In [6]:
# pip install ipython torch python-dotenv opencv-python matplotlib roboflow ultralytics mss numpy pyautogui pillow transformers easyocr

from IPython import display
import torch
from dotenv import load_dotenv
import shutil
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import roboflow
import ultralytics
from ultralytics import YOLO
import mss
import numpy as np
import time
import pyautogui
from datetime import datetime
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers import AutoTokenizer
import easyocr

WARNING Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Alice Dias\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [7]:
display.clear_output()
ultralytics.checks()

Ultralytics 8.3.133  Python-3.11.9 torch-2.5.1+cpu CPU (AMD Ryzen 5 3600 6-Core Processor)
Setup complete  (12 CPUs, 15.9 GB RAM, 236.7/931.5 GB disk)


In [8]:
# Em apple sillicon, verificar se o MPS está disponível
print(torch.backends.mps.is_available())

False


In [9]:
# Carregar variáveis do ficheiro .env
load_dotenv(override=True)

# Obter variáveis
api_key = os.getenv("ROBOFLOW_API_KEY")
workspace = os.getenv("ROBOFLOW_WORKSPACE")
project_name = os.getenv("ROBOFLOW_PROJECT")
version = int(os.getenv("ROBOFLOW_VERSION", "1"))

In [10]:
ocr = easyocr.Reader(['en'], gpu=False)  # Define idioma e se quer usar GPU

Using CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
# descarregar o dataset do roboflow, depois de etiquetadas as imagens e criado o dataset em Mac tive problemas de permissões e foi necessário dar permissões à pasta de conf. do roboflow:
# - sudo mkdir /Users/davidecarneiro/.config/roboflow (criar pasta onde vai guardar a conf.)
# - sudo chown -R davidecarneiro:staff ~/.config/roboflow (dar permissões ao meu user)
roboflow.login()

rf = roboflow.Roboflow(api_key=api_key) # Faz login no Roboflow com a chave lida

# substituir nome do workspace e do projeto
project = rf.workspace(workspace).project(project_name)
# se versão do dataset > 1, substituir pela versão correspondente
dataset = project.version(version).download("yolov8")

# Move o dataset para a pasta desejada
shutil.move(dataset.location, "./datasets/candy-crush-saga-v3-1")

# Atualiza o caminho no objeto
dataset.location = "./datasets/candy-crush-saga-v3-1"

# WARN: necessário verificar os paths no ficheiro data.yaml, após este ser descarregado
# alterar para:
# train: ./datasets/candy-crush-saga-v3-1/train/images
# val: ./datasets/candy-crush-saga-v3-1/valid/images
# test: ./datasets/candy-crush-saga-v3-1/test/images

In [ ]:
# treinar o modelo
# lista de modelos pre-treinados disponível em https://docs.ultralytics.com/models/yolov8/#performance-metrics
model = YOLO("YOLOv8m.pt")  # carregar o modelo pre-treinado que se descarregou

# Treinar o modelo
results = model.train(data='./datasets/candy-crush-saga-v3-1/data.yaml', epochs=100, imgsz=640, device='cpu')  # windows amd cpu and gpu
# results = model.train(data='candy-crush-saga-v3-1', epochs=100, imgsz=640, device=[0, 1]) # intel/windows
# results = model.train(data='candy-crush-saga-v3-1/data.yaml', epochs=100, imgsz=640, device='mps') # apple sillicon

In [3]:
# selecionar a melhor versão do modelo fine-tuned
model = YOLO("runs/detect/train5/weights/best.pt")

In [ ]:
# prever em novas imagens
confidence_level = 0.1
input_path = 'captured_images'
output_path = 'detections'
class_names = model.names

for file in os.listdir(input_path):
    if file.lower().endswith((".png")):
        image = cv2.imread(os.path.join(input_path, file))
        results = model.predict(source=image, conf=confidence_level)  # gerar previsões acima de determinada confiança, e guardar imagens


        output_filename = f"prediction_{file}"
        output_filepath = os.path.join(output_path, output_filename)

        for result in results:
            result.save(filename=output_filepath)
            print("==== Resultados Previsão ====")
            print("Imagem: "+os.path.join(input_path, file))
            boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes (x_min, y_min, x_max, y_max)
            scores = result.boxes.conf.cpu().numpy()  # Score de confiança
            labels = result.boxes.cls.cpu().numpy()  # Índice da classe

            for i in range(len(boxes)):
                class_id = labels[i]
                class_label = class_names[class_id] if class_id in class_names else "Desconhecido"

                print(f"--- Objeto {i+1} ---")
                print(f"Class: {class_label} (ID: {class_id})")
                print(f"Coordenadas Bounding Box: {boxes[i]}")
                print(f"Confiança: {scores[i]:.4f}")
                print("-------------------")

            print("\n")

# 🎮Testar no jogo, o modelo

###### CONFIGURAÇÕES GERAIS

In [11]:
model = YOLO("runs/detect/train5/weights/best.pt") # Carrega o modelo YOLO treinado
screen_width, screen_height = pyautogui.size() # Tamanho total do ecrã (para eventualmente converter coordenadas para cliques)

# Região do tabuleiro (ajustável via variáveis de ambiente)
BOARD_REGION = {
    "left": int(os.getenv("BOARD_REGION_LEFT", "0")),     # posição X do canto superior esquerdo
    "top": int(os.getenv("BOARD_REGION_TOP", "0")),      # posição Y do canto superior esquerdo
    "width": int(os.getenv("BOARD_REGION_WIDTH", "0")),    # largura da região
    "height": int(os.getenv("BOARD_REGION_HEIGHT", "0"))    # altura da região
}

# Número de linhas e colunas no tabuleiro (ex: 7x7)
ROWS = int(os.getenv("ROWS_GRID", "8"))
COLS = int(os.getenv("COLS_GRID", "8"))

# Região do objetivo (ajustável via variáveis de ambiente)
GOAL_REGION = {
    "left": int(os.getenv("GOAL_REGION_LEFT", "0")),     # posição X do canto superior esquerdo
    "top": int(os.getenv("GOAL_REGION_TOP", "0")),      # posição Y do canto superior esquerdo
    "width": int(os.getenv("GOAL_REGION_WIDTH", "0")),    # largura da região
    "height": int(os.getenv("GOAL_REGION_HEIGHT", "0"))    # altura da região
}

# Garante que as pastas de output existem
os.makedirs("captured_images", exist_ok=True)
os.makedirs("captured_goals", exist_ok=True)
os.makedirs("detections", exist_ok=True)
os.makedirs("detectionsGoal", exist_ok=True)

###### FUNÇÃO DE CAPTURA DE IMAGEM

In [12]:
# função que captura só a região do tabuleiro e devolve a imagem
def capture_board_region():
    # Captura a imagem da região definida como o tabuleiro
    with mss.mss() as sct:
        screenshot = sct.grab(BOARD_REGION)
        img = np.array(screenshot)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        img_height, img_width, _ = img.shape
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S-%f")

        # Guardar imagem opcionalmente
        img_path = os.path.join('captured_images', f"capture_{timestamp}.jpg")
        cv2.imwrite(img_path, img)

        return img, timestamp, img_width, img_height # width, height

###### FUNCAO DE CAPTURA DA REGIAO DOS OBJETIVOS

In [13]:
# função que captura só a região dos objetivos e devolve a imagem
def capture_goal_region():
    # Captura a imagem da região definida como os objetivos
    with mss.mss() as sct:
        screenshot = sct.grab(GOAL_REGION)
        img = np.array(screenshot)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        img_height, img_width, _ = img.shape
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S-%f")

        # Guardar imagem opcionalmente
        img_path = os.path.join('captured_goals', f"goal_{timestamp}.jpg")
        cv2.imwrite(img_path, img)

        return img, timestamp, img_width, img_height # width, height

###### IDENTIFICAR PECAS ADJACENTES E POSSIVEIS COMBINACOES

In [27]:
def sugerir_melhor_jogada(grid):
    ROWS, COLS = len(grid), len(grid[0])
    melhores_trocas = []
    max_combo = 0
    troca_com_color_bomb = None

    # Criar pasta se não existir
    os.makedirs("debug_objetivos_crop", exist_ok=True)

    # Captura zona dos objetivos
    imgGoal, timestampGoal, img_widthGoal, img_heightGoal = capture_goal_region()

    # Previsão com YOLO
    resultsGoal = model.predict(source=imgGoal, conf=0.1)
    caixas = resultsGoal[0].boxes.xyxy.cpu().numpy()
    classes = resultsGoal[0].boxes.cls.cpu().numpy()
    nomes_classes = model.names

    objetivos_detectados = []

    print(f"[DEBUG] Objetivos detetados: {objetivos_detectados}")

    for i, box in enumerate(caixas):
        class_id = int(classes[i])
        nome_doce = nomes_classes[class_id]
        print(f"[DEBUG] Detetado objetivo: {nome_doce} na posição {box.astype(int)}")

        # Verifica se o doce está presente no tabuleiro
        if any(nome_doce in row for row in grid):
            x1, y1, x2, y2 = box.astype(int)
            altura_icon = y2 - y1
            numero_crop = imgGoal[y2:y2 + int(altura_icon * 0.8), x1:x2]

            # Guardar o crop da zona onde estará o número
            nome_ficheiro = f"debug_objetivos_crop/{timestampGoal}_{nome_doce}_{i}.jpg"
            cv2.imwrite(nome_ficheiro, numero_crop)

            # OCR (pode falhar)
            resultado_ocr = ocr.readtext(numero_crop)
            numero = None
            if resultado_ocr:
                for linha in resultado_ocr:
                    print(f"[DEBUG OCR] {nome_doce} → {resultado_ocr}")
                    texto = linha[1]
                    print(f"[DEBUG texto OCR]: {texto}")
                    if texto.isdigit():
                        numero = int(texto)
                        break

            if numero is not None:
                objetivos_detectados.append((nome_doce, numero))

    # Função auxiliar
    def contar_sequencia(g, r, c, cor):
        cont_h = 1
        i = c - 1
        while i >= 0 and g[r][i] == cor:
            cont_h += 1
            i -= 1
        i = c + 1
        while i < COLS and g[r][i] == cor:
            cont_h += 1
            i += 1

        cont_v = 1
        i = r - 1
        while i >= 0 and g[i][c] == cor:
            cont_v += 1
            i -= 1
        i = r + 1
        while i < ROWS and g[i][c] == cor:
            cont_v += 1
            i += 1

        return max(cont_h if cont_h >= 3 else 0, cont_v if cont_v >= 3 else 0)

    melhor_jogada_objetivo = None
    max_prioridade = 0

    for r in range(ROWS):
        for c in range(COLS):
            if grid[r][c] is None:
                continue

            for dr, dc in [(0, 1), (1, 0)]:
                nr, nc = r + dr, c + dc
                if nr >= ROWS or nc >= COLS or grid[nr][nc] is None:
                    continue

                if grid[r][c] == "color-bomb" or grid[nr][nc] == "color-bomb":
                    troca_com_color_bomb = ((r, c), (nr, nc))

                grid[r][c], grid[nr][nc] = grid[nr][nc], grid[r][c]

                score = max(
                    contar_sequencia(grid, r, c, grid[r][c]),
                    contar_sequencia(grid, nr, nc, grid[nr][nc])
                )

                if score > max_combo:
                    max_combo = score
                    melhores_trocas = [((r, c), (nr, nc))]
                elif score == max_combo and score >= 3:
                    melhores_trocas.append(((r, c), (nr, nc)))

                for doce, _ in objetivos_detectados:
                    if grid[r][c] == doce or grid[nr][nc] == doce:
                        if score > max_prioridade:
                            max_prioridade = score
                            melhor_jogada_objetivo = ((r, c), (nr, nc))

                grid[r][c], grid[nr][nc] = grid[nr][nc], grid[r][c]

    if troca_com_color_bomb:
        print(f"\n💡 Jogada com 'color-bomb': {troca_com_color_bomb} (prioritária)")
        return troca_com_color_bomb, 99

    if melhor_jogada_objetivo:
        print(f"\n💡 Jogada com doce de objetivo: {melhor_jogada_objetivo} com prioridade {max_prioridade}")
        return melhor_jogada_objetivo, max_prioridade

    if melhores_trocas:
        print(f"\n💡 Melhor jogada: {melhores_trocas[0]} que forma grupo de {max_combo}")
        return melhores_trocas[0], max_combo

    print("\n⚠️ Nenhuma jogada possível encontrada.")
    return None, 0

###### LOOP PRINCIPAL

In [ ]:
while True:
    # Captura o tabuleiro
    img, timestamp, img_width, img_height = capture_board_region()

    # Faz previsão com YOLO
    results = model.predict(source=img, conf=0.1)
    detections = results[0].boxes.xyxy  # Coordenadas (x1, y1, x2, y2)

    # Define dimensões das células da grelha
    cell_width = img_width // COLS
    cell_height = img_height // ROWS

    grid = [[None for _ in range(COLS)] for _ in range(ROWS)]  # Cria grelha vazia


    if len(detections) > 0:
        print(f"\n✅ Detetou {len(detections)} objetos.")

        # Guarda imagem com bounding boxes
        annotated_frame = results[0].plot()
        result_path = os.path.join('detections', f"result_{timestamp}.jpg")
        cv2.imwrite(result_path, annotated_frame)

        # Processa cada deteção
        for i, (x1, y1, x2, y2) in enumerate(detections.tolist()):
            # Calcula o centro do objeto
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # Converte centro para coordenadas de grelha
            col = center_x // cell_width
            row = center_y // cell_height

            class_id = int(results[0].boxes.cls[i].item())
            class_name = model.names[class_id]

            # Preenche a grelha com o nome da classe
            if 0 <= row < ROWS and 0 <= col < COLS:
                grid[row][col] = class_name

            # converter coordenadas
            scaled_x = int((center_x / img_width) * screen_width)
            scaled_y = int((center_y / img_height) * screen_height)

            # Pausa breve, entre objetos (se necessario)
            #time.sleep(0.25)

        # Imprime o mapa atual do tabuleiro
        print("📌 Mapa do tabuleiro:")
        for row in grid:
            print(row)

        print("\n🔄 Combinações possíveis:")
        # Sugerir a melhor jogada
        melhor_troca, tamanho_grupo = sugerir_melhor_jogada(grid)

        # mover rato
        if melhor_troca:
            (r1, c1), (r2, c2) = melhor_troca

            def cell_to_screen(row, col):
                # Coordenadas do centro da célula no ecrã
                x = BOARD_REGION["left"] + col * cell_width + cell_width // 2
                y = BOARD_REGION["top"] + row * cell_height + cell_height // 2
                return x, y

            x1, y1 = cell_to_screen(r1, c1)
            x2, y2 = cell_to_screen(r2, c2)

            print(f"🖱️ A mover para ({r1}, {c1}) -> ({r2}, {c2})")
            pyautogui.moveTo(x1, y1, duration=0.3)
            pyautogui.click()
            time.sleep(0.2)  # pequena pausa entre cliques
            pyautogui.moveTo(x2, y2, duration=0.3)
            pyautogui.click()
        else:
            print("⚠️ Nenhuma jogada possível para clicar.")


    time.sleep(3) # Pausa antes da próxima leitura (ajustavel)